In [1]:
#!/usr/bin/env python
# -*- coding: latin-1 -*-
import pandas as pd # pandas is a data manipulation library
import numpy as np #provides numerical arrays and functions to manipulate the arrays efficiently
import logging
import difflib
import Levenshtein as lev
from tqdm.notebook import tqdm
import multiprocessing
import gensim.models.doc2vec
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from scipy import spatial
import re
import movies_df_helper_functions as helper_fn
import evaluation_helper_functions as eval_fn
import random
import imp
import itertools
from sklearn.manifold import TSNE
from sklearn.cluster import AffinityPropagation
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

from tabulate import tabulate
import decimal

import csv
import imp
imp.reload(eval_fn)
imp.reload(helper_fn)
%matplotlib inline

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)
decimal.getcontext().rounding = decimal.ROUND_DOWN
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
tqdm.pandas()

d:\programs\python39\lib\site-packages\tqdm\std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
# load movies.csv
movies_df = pd.read_csv('generated_dataset/yago_pedia_ml_final_movies_list.csv')

In [4]:
movies_df.replace('NOT_FOUND',np.nan, inplace=True)

In [5]:
def isNaN(string):
    return string != string

In [6]:
def data_processing(df, col_name):    
    regex = "\(.*?\)"
    df[col_name] = df[col_name].apply(lambda 
                            row: re.sub(regex,'',row) if isNaN(row)== False  else row)
    df[col_name] = df[col_name].apply(lambda row: 
                            row.lower().strip().replace(' ','_').split('|') if isNaN(row)== False  else row)
    df[col_name] = df[col_name].apply(lambda 
                            row: [item.strip() for item in row] if isNaN(row)== False  else row)
    return df

In [7]:
movies_df = data_processing(movies_df, 'actor_name')
movies_df = data_processing(movies_df, 'writer_name')
movies_df = data_processing(movies_df, 'genre')
movies_df = data_processing(movies_df, 'country')
movies_df = data_processing(movies_df, 'director_name')
movies_df.describe

<bound method NDFrame.describe of                      movie_id                   title  \
0                       $9.99                   $9.99   
1                  '68_(film)                     '68   
2                  '71_(film)                     '71   
3                 'Gator_Bait             'Gator Bait   
4      'Neath_Brooklyn_Bridge  'Neath Brooklyn Bridge   
...                       ...                     ...   
36353                    Zozo                    Zozo   
36354                Zubeidaa                Zubeidaa   
36355        Zulu_(1964_film)                    Zulu   
36356        Zulu_(2013_film)                    Zulu   
36357                Zus_&_Zo                Zus & Zo   

                                              actor_name  \
0                      [geoffrey_rush, anthony_lapaglia]   
1                                [jan_němec, neil_young]   
2      [richard_dormer, charlie_murphy_, sean_harris,...   
3                                     [cl

In [ ]:
movies_df

In [8]:
movies_df['movie_id']=movies_df['movie_id'].str.lower()
movies_df['title']=movies_df['title'].str.lower()
movies_df[['movie_id','title']]

,movie_id,title
0,$9.99,$9.99
1,'68_(film),'68
2,'71_(film),'71
3,'gator_bait,'gator bait
4,'neath_brooklyn_bridge,'neath brooklyn bridge
...,...,...
36353,zozo,zozo
36354,zubeidaa,zubeidaa
36355,zulu_(1964_film),zulu
36356,zulu_(2013_film),zulu


In [9]:
  
regex = "\(.*?\)"
df[col_name] = df[col_name].apply(lambda row: re.sub(regex,'',row) if isNaN(row)== False  else row)
df[col_name] = df[col_name].apply(lambda row: 
                            row.lower().strip().replace(' ','_').split('|') if isNaN(row)== False  else row)
df[col_name] = df[col_name].apply(lambda row: [item.strip() for item in row] if isNaN(row)== False  else row)

NameError: name 'df' is not defined

# Create & Train doc2vec Model

**Using movies from the dataset where non of the features is missing

*** Choose the train data ***

In [10]:
train_data_df =movies_df[~(movies_df.actor_name.isnull())
                          & ~(movies_df.director_name.isnull())
                          & ~(movies_df.writer_name.isnull())
                          & ~(movies_df.genre.isnull())
                          & ~(movies_df.country.isnull())]
train_data_df.reset_index(drop= True, inplace=True)
#train_data_df.to_csv("models/baseline_exp/train_data.csv", index= False)
#train_data_df.describe

***Build movie sentence and create taggeddocuments ***

In [11]:
movies_df.drop(movies_df.loc[movies_df.movie_id=='suburbia_(film)'].index, inplace= True)



In [12]:
def build_movie_sentence(actor,director,writer,genre,country):
    return list(set(actor+director+writer+genre+country))

In [13]:
train_data_df ['movie_sentence']= train_data_df.progress_apply(lambda row:  build_movie_sentence(row.actor_name, row.director_name, row.writer_name, row.genre, row.country), axis=1)
train_data_df.to_csv("models/train_data/train_data.csv", index=False)

<ipython-input-13-9fed7ca8e9c4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_df ['movie_sentence']= train_data_df.progress_apply(lambda row:  build_movie_sentence(row.actor_name, row.director_name, row.writer_name, row.genre, row.country), axis=1)


FileNotFoundError: [Errno 2] No such file or directory: 'models/train_data/train_data.csv'

In [14]:
#create dictionary with the movie_id and title
movies_dict= dict(zip(train_data_df.movie_id, train_data_df.title))
movies_dict


{"'68_(film)": "'68",
 "'71_(film)": "'71",
 "'gator_bait": "'gator bait",
 "'neath_the_arizona_skies": "'neath the arizona skies",
 "'r_xmas": "'r xmas",
 "'til_there_was_you": "'til there was you",
 "'twas_the_night": "'twas the night",
 "'twas_the_night_before_christmas_(1977_tv_special)": "'twas the night before christmas",
 '...ing': '...ing',
 '.45_(film)': '.45',
 '008:_operation_exterminate': '008: operation exterminate',
 '10,000_bc_(film)': '10,000 bc',
 '100%25_love_(2011_film)': '100% love',
 '1001_grams': '1001 grams',
 '100_bloody_acres': '100 bloody acres',
 '100_days_of_love': '100 days of love',
 '100_feet': '100 feet',
 '100_ghost_street:_the_return_of_richard_speck': '100 ghost street: the return of richard speck',
 '100_girls': '100 girls',
 '100_rifles': '100 rifles',
 '100_tears': '100 tears',
 '101_dalmatians_(1996_film)': '101 dalmatians',
 '102_dalmatians': '102 dalmatians',
 '10_(film)': '10',
 '10_cloverfield_lane': '10 cloverfield lane',
 '10_items_or_less_(

In [15]:
# Create the tagged document needed for Doc2Vec-=
tagged_documents=[]
train_data_df.progress_apply(lambda row : tagged_documents.append(TaggedDocument(row.movie_sentence,[row.movie_id])),axis=1)

#convert the series of tagged docuemnt to a list
train_data = list(tagged_documents)

print("Length of training data {}".format(len(train_data)))


Length of training data 19492


In [ ]:
random.seed = 400
[random.shuffle(data.words) for data in train_data]

***Configure and train the model ***

In [31]:
cores = multiprocessing.cpu_count()
model = gensim.models.doc2vec.Doc2Vec(dm= 0, dbow_words=1, vector_size= 100, 
                                      hs=0 ,min_count=5,
                                      workers=cores, epochs= 50,
                                      ns_exponent=(-3), negative= 20, window_size= 1000000)
model.build_vocab(train_data)
model.train(train_data, total_examples=len(train_data) , epochs=model.epochs)

2020-12-07 18:38:41,071 : INFO : collecting all words and their counts
2020-12-07 18:38:41,073 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-12-07 18:38:41,120 : INFO : PROGRESS: at example #10000, processed 95094 words (1992256/s), 29478 word types, 10000 tags
2020-12-07 18:38:41,176 : INFO : collected 43764 word types and 19491 unique tags from a corpus of 19492 examples and 185880 words
2020-12-07 18:38:41,177 : INFO : Loading a fresh vocabulary
2020-12-07 18:38:41,302 : INFO : effective_min_count=5 retains 6620 unique words (15% of original 43764, drops 37144)
2020-12-07 18:38:41,302 : INFO : effective_min_count=5 leaves 129179 word corpus (69% of original 185880, drops 56701)
2020-12-07 18:38:41,324 : INFO : deleting the raw counts dictionary of 43764 items
2020-12-07 18:38:41,326 : INFO : sample=0.001 downsamples 32 most-common words
2020-12-07 18:38:41,327 : INFO : downsampling leaves estimated 97415 word corpus (75.4% of prior 129179)
2020

In [32]:
model.save('../expirements/exp3/model')

2020-12-07 21:13:40,270 : INFO : saving Doc2Vec object under ../expirements/exp3/model, separately None
2020-12-07 21:13:40,659 : INFO : saved ../expirements/exp3/model


In [34]:
len(train_data)

19492

In [28]:
#get the words vectors represenation
words_vectors = model.wv
#get the docs vectors represenation
docs_vectors = model.docvecs

In [29]:
print(words_vectors.similarity("adam_sandler", "jennifer_aniston"))
print(words_vectors.similarity('douglas_hickox','cy_endfield'))

0.5224008


KeyError: "word 'douglas_hickox' not in vocabulary"

In [40]:
val = input("Enter your value: ") 
print(val) 

Enter your value:  toy story


toy story


In [38]:
sim = docs_vectors.most_similar(positive= 'toy_story', topn= 5)
eval_fn.print_sim_list(sim)

TypeError: '<' not supported between instances of 'str' and 'int'

In [37]:
sum(docs_vectors['man_of_the_year_(2006_film)']- docs_vectors['mean_girls'])

0.2068108143284917

In [ ]:
def print_sim_list(sim_list):
    print(tabulate(sim_list, headers= ['Movie ID', 'Similarity']))
    

In [ ]:
m = 'scrooge_(1951_film)'
sim_list = docs_vectors.most_similar(positive= m, topn= 5)
results = calculate_BLEU_score(m, sim_list, train_data_df)



In [ ]:
x = train_data_df[train_data_df['movie_id'].str.lower() == 'harry_potter_and_the_prisoner_of_azkaban_(film)'.lower()].movie_sentence
x

In [ ]:
m1 = helper_fn.find_movie_sent_by_id('oliver_twist_(1948_film)',train_data_df)
m2 = helper_fn.find_movie_sent_by_id('harry_potter_and_the_chamber_of_secrets_(film)', train_data_df)
smoothie = SmoothingFunction().method4
sentence_bleu([m1], m2, smoothing_function=smoothie, weights=(1,0,0,0))

In [ ]:
jaccard(set(m1), set(m2))

In [ ]:
from jiwer import wer
import jiwer

In [ ]:
error = wer(m1, m2)
print(error)

In [ ]:
jiwer.wer(m1,m2)
jiwer.mer(m1,m2)
jiwer.wil(m1,m2)

In [ ]:
from collections import Counter
import math

m1 = helper_fn.find_movie_sent_by_id('harry_potter_and_the_prisoner_of_azkaban_(film)',train_data_df)
m2 = helper_fn.find_movie_sent_by_id('harry_potter_and_the_chamber_of_secrets_(film)', train_data_df)

counterA = Counter(m1)
counterB = Counter(m2)


def counter_cosine_similarity(c1, c2):
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)

print(counter_cosine_similarity(counterA, counterB))

In [ ]:
x1 = helper_fn.find_movie_sent_by_id('harry_potter_and_the_prisoner_of_azkaban_(film)',train_data_df)
x2 = helper_fn.find_movie_sent_by_id('harry_potter_and_the_chamber_of_secrets_(film)',train_data_df)

def jaccard(a, b):
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

jaccard(set(x1), set(x2))

In [ ]:
eval_fn.find_common_attrs(train_data_df, 'harry_potter_and_the_prisoner_of_azkaban_(film)','harry_potter_and_the_chamber_of_secrets_(film)')

In [ ]:
sent = helper_fn.find_movie_sent_by_id('the_hunchback_of_notre_dame_(1911_film)',train_data_df)
print(sent[0])
s= ['tobey_aguire','kirsten_dunst']

infered_movie = model.infer_vector(s)
print(infered_movie)
similar_to_infered = docs_vectors.most_similar([infered_movie], topn=20)

print(*similar_to_infered, sep='\n')

In [ ]:
def start_param_search(train_data_df):
    #prepare train
    tagged_documents=[]
    train_data_df.progress_apply(lambda row : tagged_documents.append(TaggedDocument(row.movie_sentence,[row.movie_id])),axis=1)
    train_data = list(tagged_documents)
    random.seed = 400
    [random.shuffle(data.words) for data in train_data]
    random.shuffle(train_data)
    
    #prepare test data
    test_titles = []
    test_titles= ['shrek','star trek','toy story','blade','pitch perfect','harry potter','godzilla','paddington','x-men']
    test_data= eval_fn.build_movie_sequel_test_data(train_data_df, test_titles)
    
    #prepare parameters list
    top=5
    dm = [0]
    vector_size = [100,300]
    window = [100000]
    hs = [0]
    min_count=[1,5]
    epochs=[50]
    ns_exponent=[3,-3]
    dbow_words=[1]
    negative=[5,20]
    param_list = [{'dm': item[0],'vector_size': item[1],'window': item[2],'hs': item[3],'epochs':item[4], 'min_count':item[5],'ns_exponent':item[6], 'dbow_words':item[7],'negative':item[8] }
                  for item in
                         list(itertools.product(*[dm, vector_size, window, hs, epochs, min_count, ns_exponent, dbow_words, negative]))
                ]
    print(len(param_list))
    #print(len(param_list))
    models_scores = eval_fn.model_parameters_search (train_data, test_data, param_list, train_data_df, top)

    score_doc2vec = pd.DataFrame(models_scores)
    score_doc2vec = score_doc2vec.sort_values(by=['rank_score'],
                                             ascending=False)
    return score_doc2vec
    #call the search function
    

In [ ]:
imp.reload(eval_fn)
eval_score_df = start_param_search(train_data_df)

In [ ]:
eval_score_df

In [ ]:
save_eval_score_df = eval_score_df.copy()
save_eval_score_df.drop(columns= ['model'],inplace= True)

In [ ]:
save_eval_score_df.to_excel('models/grid_search_exp/GS_2.xlsx', index=False)


In [ ]:
def build_movie_sequel_test_data(train_df, titles_list):
    test_data_list = []
    for title in titles_list:
        
        movies_list = find_movie_sequel(train_df, title)
        if movies_list is not None:
            test_data_list.append(movies_list)
    return test_data_list   

def find_movie_sequel(df, movie_name):
    movie_name_regex= '^' + movie_name + "(.*)$"
    #print(movie_name)
    #print(df)
    matched_movies = df[df['title'].str.match(movie_name_regex.lower())]
    return matched_movies.movie_id.values.tolist() if len(matched_movies) >0 else None


In [ ]:
ranks, second_ranks = calculate_self_similarity(model, train_data[0:100])

In [ ]:
counter = collections.Counter(second_ranks)
print(counter)

In [ ]:
def construct_titles_list(titles):
    return [title for title in titles]

sequel_titles =[train_data_df[train_data_df.title.str.match('^shrek(.*)$')]['movie_id'],
train_data_df[train_data_df.title.str.match('^star trek(.*)$')]['movie_id'],
train_data_df[train_data_df.title.str.match('^toy story(.*)$')]['movie_id'],
train_data_df[train_data_df.title.str.match('^the avengers(.*)$')]['movie_id'],
train_data_df[train_data_df.title.str.match('^iron man(.*)$')]['movie_id'],
train_data_df[train_data_df.title.str.match('^cars(.*)$')]['movie_id'],
train_data_df[train_data_df.title.str.match('^blade(.*)$')]['movie_id'],
train_data_df[train_data_df.title.str.match('^pitch perfect(.*)$')]['movie_id'],
train_data_df[train_data_df.title.str.match('^harry potter(.*)$')]['movie_id'],
train_data_df[train_data_df.title.str.match('^godzilla(.*)$')]['movie_id'],
train_data_df[train_data_df.title.str.match('^paddington(.*)$')]['movie_id'],
train_data_df[train_data_df.title.str.match('^spider(.*)$')]['movie_id'],
train_data_df[train_data_df.title.str.match('^x-men(.*)$')]['movie_id']]
def construct_titles_list(titles):
    return [title for title in titles]
for sequel in sequel_titles:
    test_titles= test_titles + construct_titles_list(sequel)

test_titles = test_titles + doc_tags[5000:5050]
tsne_plot_doc2vec(model, test_titles, movies_dict)

In [ ]:
train_data_df['genre'].unique

In [ ]:
m1 = helper_fn.find_movie_id_by_title('truck turner',train_data_df)
m2 = helper_fn.find_movie_id_by_title('stealth fighter',train_data_df)
#print(m1,m2)
print(m1[0])
print(m2[0])
x = model[m1[0]] - model[m2[0]]
print(sum(x))

In [ ]:
m1 = helper_fn.find_movie_id_by_title('the girl in the red velvet swing',train_data_df)
m2 = helper_fn.find_movie_id_by_title('the thirteenth guest',train_data_df)
#print(m1,m2)
print(m1[0])
print(m2[0])
x = model[m1[0]] - model[m2[0]]
print(sum(x))

In [ ]:
doc_tags = list(model.docvecs.doctags.keys())
sample_tags= doc_tags[0:1000]
sample_docs = model[sample_tags]

In [ ]:
movies_dict

In [ ]:
def tsne_plot_doc2vec(model, samples,movies_dict):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []
    

    for doc in samples:
        tokens.append(model[doc])
        labels.append(movies_dict[doc])
    
    tsne_model = TSNE(perplexity=50, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    #plt.xlim(-4.0, -1.0)
    plt.show()

In [ ]:
sample_tags = ['harry_potter_and_the_chamber_of_secrets_(film)',
'harry_potter_and_the_prisoner_of_azkaban_(film)',
'harry_potter_and_the_goblet_of_fire_(film)', 
'harry_potter_and_the_half-blood_prince_(film)',
'harry_potter_and_the_order_of_the_phoenix_(film)']
doc_tags = list(model.docvecs.doctags.keys())
bad_sample = doc_tags[100:130]
samples = set(sample_tags + bad_sample)
tsne_plot_doc2vec(model, samples)

In [ ]:
x = model['harry_potter_and_the_chamber_of_secrets_(film)'] - model['harry_potter_and_the_prisoner_of_azkaban_(film)']
print(x)
sum(x)

In [ ]:
x = model['harry_potter_and_the_chamber_of_secrets_(film)'] - model['just_go_with_it']
sum(x)

In [ ]:
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(sample_docs)
df = pd.DataFrame(X_tsne, index= sample_tags, columns=['x', 'y'])

In [ ]:
#the_bride_wore_red
m1 = helper_fn.find_movie_sent_by_id("the_beautiful_person", train_data_df)
m2 = helper_fn.find_movie_sent_by_id('second_fiddle_(1957_film)', train_data_df)

In [ ]:
print(m1)
print(m2)

In [ ]:
tsne_d2v

plt.scatter(X_tsne[:, 0], X_tsne[:, 1])
plt.show()

In [ ]:
def tsne_plot_word2vec(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []
    

    for word in model.wv.vocab[0:5000]:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [ ]:
movies_df.isnull().any()

In [ ]:
movies_df.describe

In [ ]:
def find_in_movie_sent(search_list, row):
 #   print(row)
    if all(elem in row['movie_sentence']  for elem in search_list):
        return(row['movie_id'])
    else:
        return False
        
search_list=[ 'adventure', 'family','chris_columbus', 'j.k._rowling']
x=movies_df.apply(lambda row: find_in_movie_sent(search_list, row),axis=1)
#doc_tags =[]
for item in x:
    if item != False:
        doc_tags.append(item)
        

    

In [ ]:
doc_tags

In [ ]:
tsne_plot(list(set(tags)),model)
#docs_vectors.doctags

In [ ]:
#model['going_overboard_tt0096870']
sim = docs_vectors.most_similar([model['billy_madison_tt0112508']])

In [ ]:
tags=[] 
for id, similarity in sim:
    tags.append(id)

In [ ]:
print(*doc_tags, sep='\n')

In [ ]:
movie_sent_dict = dict(zip(movies_df.movie_id, movies_df.movie_sentence))

In [ ]:
target_id = 'the_hunger_games_tt1392170'
n_items= 5
treshold=.4
data_size= 10

topn_list = model.docvecs.most_similar([target_movie_id], topn= n_items + 1)
#remove the target movie from the list
for idx, sim in enumerate(topn_list):    
    if sim[0] == target_id:        
        topn_list.pop(idx)
#(topn_list.pop(idx) if (topn_list[0] == target_movie_id ) for idx, sim in enumerate(topn_list))
precision_df = calculate_precision(target_id, topn_list, movie_sent_dict, treshold)

In [ ]:
model.docvecs.most_similar([model[target_id]], topn=5+1)

In [ ]:
model.docvecs.most_similar([target_id], topn=5)

In [ ]:
random.seed = 400
n_items= 5
treshold=.44
data_size= 100
random.seed = 150
dict_keys = list(movie_sent_dict.keys())
random.shuffle(dict_keys)
movie_id_list = random.sample(dict_keys, data_size)
#movie_id_list=["the_hunger_games_tt1392170", "harry_potter_and_the_sorcerer's_stone_tt0241527"]
precision_df = calculate_precision_data_sample(5 , 4 ,movie_id_list, movie_sent_dict, treshold )
#movie_id_list

precision_df

In [ ]:
docs_vectors.most_similar(['harry_potter_and_the_sorcerer\'s_stone_tt0241527'])
docs_vectors.most_similar(['survivor_tt3247714'])

In [ ]:


movies_df.sample(frac=1)

In [ ]:
sample_movies_df = train_data_df.loc[train_data_df.apply(lambda movie: True if ('romantic_comedy' in movie['genre']) 
                                                 and ('adam_sandler' in movie.actor_name) else False, axis=1)]

In [ ]:
sample_movies_df

In [ ]:
sample_movies_df[['title','genre','actor_name','director_name','writer_name']]

In [ ]:
#drama_movies_df = movies_df.loc[movies_df.genre.apply(lambda genre: True if 'drama' in genre else False)]

common_attrs_df= pd.DataFrame(columns=['movie_id','actors_cnt', 'directors_cnt','writers_cnt','genres_cnt','total_cnt'])

sample_movies_df.reset_index(drop=True, inplace=True)
print(query_movie_id)
query_movie_id= sample_movies_df.iloc[2].movie_id

common_attrs_df= common_attrs_df.append(sample_movies_df.progress_apply(lambda row: find_common_attrs(sample_movies_df,query_movie_id, row.movie_id), axis=1)
                                        ,ignore_index = True)

print(common_attrs_df)

In [ ]:
sample_movies_df

In [ ]:
sample_movies_df = train_data_df.loc[train_data_df.apply(lambda movie: True if ('romantic_comedy' in movie['genre']) 
                                                      and ('adam_sandler' in movie.actor_name) else False, axis=1)]

cols=['actor_name','director_name', 'writer_name','genre','country']
common_attr_matrix = sample_movies_df.apply(lambda row: eval_fn.find_common_attrs_in_list(sample_movies_df, row.movie_id, cols), axis=1)
common_attr_matrix.set_index('title', inplace= True)

In [ ]:
common_attr_matrix['Title'] = common_attr_matrix.Title.str.capitalize()
common_attr_matrix.columns = common_attr_matrix.columns.str.capitalize()

In [ ]:
common_attr_matrix

In [ ]:
sample_movies_df = movies_df.loc[movies_df.apply(lambda movie: True if ('romance' in movie['genre']) 
                                                 and ('comedy' in movie.genre) and ('adam_sandler' in movie.actor_name) else False, axis=1)]

In [ ]:
common_attrs_df = common_attrs_df[common_attrs_df.movie_id != query_movie_id]

In [ ]:
common_attrs_df[common_attrs_df.total_cnt == common_attrs_df.total_cnt.max()]


In [ ]:
docs_vectors.most_similar('the_wedding_singer_tt0120888', topn=30)

In [ ]:
helper_fn.find_movie_sent_by_id('a_night_at_the_roxbury_tt0120770',movies_df)

In [ ]:
helper_fn.find_movie_sent_by_id('the_wedding_singer_tt0120888',movies_df)

In [ ]:
helper_fn.find_movie_sent_by_id('mr._deeds_tt0280590',movies_df)

In [ ]:
find_common_attrs(movies_df,'the_wedding_singer_tt0120888', 'mr._deeds_tt0280590')

In [ ]:
find_common_attrs(movies_df,'the_wedding_singer_tt0120888', 'a_night_at_the_roxbury_tt0120770')

In [ ]:
def find_common_attrs_dict(movies_df,query_movie_id, search_movie_id):
    query_movie= movies_df[movies_df.movie_id == query_movie_id]
    search_movie= movies_df[movies_df.movie_id == search_movie_id]
    
    common_actors= count_sim(query_movie, search_movie, 'actor_name')
    common_directors= count_sim(query_movie, search_movie, 'director_name')
    common_writers= count_sim(query_movie, search_movie, 'writer_name')
    common_genres= count_sim(query_movie, search_movie, 'genre')
    
    common_dict = {search_movie_id: common_actors+common_directors+common_writers+common_genres}
    return common_dict

In [ ]:
    common_attrs_dict=[]
    common_attrs_dict = [movies_df.progress_apply(lambda row: find_common_attr(movies_df,query_movie_id, row.movie_id), axis=1)]

In [ ]:
drama_movies_df = movies_df.loc[movies_df.genre.apply(lambda genre: True if 'drama' in genre else False)]

In [ ]:
len(drama_movies_df)

In [ ]:
x = train_data_df.apply(lambda row: len(row.actor_name) + len(row.director_name)+len(row.writer_name)+len(row.genre)+len(row.country), axis=1)
y = train_data_df.apply(lambda row: len(row.movie_sentence) ,axis=1)
df=pd.DataFrame(data=[x,y])
df

In [ ]:
data_slice= search_list[start:end] 
data_chunck =[]
print("Searching by movie title")
data_chunck = [movie.strip().lower() for movie in data_slice]
final_search_list = '\"""' + '\""",\"""'.join(data_chunck) + '\"""' 
try:
    results = execute_query_for_movies_list(query,final_search_list,endpoint, return_format,request_method)              
    all_results_df = all_results_df.append(get_sparql_dataframe(results))  
except Exception as ex:
    print(ex)
    print("Chunck Failed : {}".format(final_search_list))


In [ ]:
m='just_go_with_it'
for doc in tagged_documents:
    if m in doc.tags:
        d = doc
        
d